# Transfering Data to Snowflake

In [ ]:
import os
import json
from pathlib import Path
import geopandas as gpd
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType

params = json.loads(os.environ.get("SNOWFLAKE_CONNECTION"))
session = Session.builder.configs(params).create()

In [29]:
GEOJSON_PATH = "ODOT_Data/dim_signal_XD.geojson"
TABLE = "XD_GEOM"

gdf = gpd.read_file(GEOJSON_PATH)
# ensure lon/lat WGS84
gdf = gdf.to_crs(epsg=4326)
# convert geometry to WKT for sql query
gdf['WKT'] = gdf['geometry'].apply(lambda g: g.wkt)
gdf = gdf[['XD', 'WKT']]
gdf

,XD,WKT
0,1236851456,POLYGON ((-123.84950293892057 46.1909069870806...
1,1236985053,POLYGON ((-123.17054104678215 44.1032762677248...
2,448854484,POLYGON ((-124.05812470345623 44.6346007743394...
3,484487649,POLYGON ((-123.19796626496156 45.2122194198096...
4,385980703,POLYGON ((-123.34827999811587 42.4224262637362...
...,...,...
16063,448855840,POLYGON ((-123.04214614172004 44.0768505441109...
16064,1236852335,POLYGON ((-122.83289018085857 45.4476262638553...
16065,448835420,POLYGON ((-122.82601626463054 45.4608324528451...
16066,385766005,POLYGON ((-122.56006557258574 45.4247273686460...


In [32]:
# prepare data
data = [(int(row['XD']), row['WKT']) for _, row in gdf.iterrows()]
schema = StructType([
    StructField("XD", IntegerType()),
    StructField("WKT", StringType())
])

# create Snowpark DataFrame
df = session.create_dataframe(data, schema)

# insert with TO_GEOGRAPHY
from snowflake.snowpark import functions as F
df = df.with_column("GEOM", F.sql_expr("TO_GEOGRAPHY(WKT)")).drop("WKT")
df.write.save_as_table("XD_GEOM", mode="truncate") # overwrite existing table